# Multi-Agent Systems
In a multi-agent system individual agents 
 - with different personas, 
 - set of tools, 
 - communicate, 
 - collaborate, 
 - add additional context, 
 - and do work in parallel. 

They often have special rules for how to work as a team, decide things together, and handle disagreements.

# Install Packages

In [ ]:
#Install packages
%pip install pyautogen
%pip install openai
%pip install chroma
%pip install python-dotenv
%pip install markdownify
%pip install apify_client
%pip install duckduckgo-search

# Initialization 

In [ ]:
import os
import autogen
from dotenv import load_dotenv

load_dotenv()
# Save your OPENAI_API_KEY in a .env file in the same directory as this script
llm_config_dict = {"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}],"cache_seed": 42, "temperature": 0}

# Two-agent chat: 
 - The simplest form of conversation pattern where two agents chat with each other using Autogen.

<img src="./images/autogen_image1.png" alt="GroupChat Flow" width="500" style="display: block; margin-left: auto; margin-right: auto;">


In [13]:
import os

from autogen import ConversableAgent

student_agent = ConversableAgent(
    name="Student_Agent",
    system_message="You are a student willing to learn. You ask meaningful and precise follow up questions and are eager to learn more.",
    llm_config=llm_config_dict,
)
teacher_agent = ConversableAgent(
    name="Teacher_Agent",
    system_message="You are an expert at Large Language model research and you teach cutting edge AI development technologies.",
    llm_config=llm_config_dict,
)

chat_result = student_agent.initiate_chat(
    teacher_agent,
    message="How to prove or test that Artificial Intelligence is actually intelligent?",
    summary_method="reflection_with_llm",
    max_turns=2,
)

Student_Agent (to Teacher_Agent):

How to prove or test that Artificial Intelligence is actually intelligent?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Teacher_Agent (to Student_Agent):

Proving or testing that an artificial intelligence (AI) system is genuinely intelligent is a complex and multifaceted challenge. Here are several approaches and methodologies that can be used to evaluate AI intelligence:

### 1. Turing Test
**Concept**: Proposed by Alan Turing in 1950, the Turing Test evaluates a machine's ability to exhibit intelligent behavior indistinguishable from that of a human.

**Method**: A human evaluator interacts with both a machine and a human through a text-based interface. If the evaluator cannot reliably distinguish the machine from the human, the machine is said to have passed the test.

**Limitations**: The Turing Test focuses on imitation rather than understanding or reasoning. It also does not acc

Let's look at the summary. It is contained within the chat_result object of type ChatResult, which was returned by the initiate_chat method.

In [ ]:
print(chat_result.summary)

In the above example, the summary method is configured to `reflection_with_llm`, which processes a list of conversation messages and summarizes them by invoking an LLM.

Initially, the summary method attempts to utilize the recipient’s LLM; if it is unavailable, it defaults to the sender’s LLM.

Here, the recipient is “Teacher_Agent” and the sender is “Student_Agent”.

The input prompt for the LLM is the following default prompt:

In [ ]:
print(ConversableAgent.DEFAULT_SUMMARY_PROMPT)

# Sequential chat: 
 - a series of conversations between two agents, linked by a carryover mechanism that transfers the summary of the prior chat to the context of the subsequent chat.

<img src="./images/autogen_image2.png" alt="GroupChat Flow" width="500" style="display: block; margin-left: auto; margin-right: auto;">


In [ ]:
from autogen import ConversableAgent

report_agent = ConversableAgent(
    name="Report_Agent",
    system_message="You are responsible for creating a report by extracting insights from the chat history.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Researcher Agent discovers the potential of AGI.
researcher_agent = ConversableAgent(
    name="Researcher_Agent",
    system_message="You explore and describe the potential capabilities and advancements of AGI.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Ethicist Agent evaluates the ethical implications of AGI.
ethicist_agent = ConversableAgent(
    name="Ethicist_Agent",
    system_message="You evaluate the ethical implications of AGI based on the research findings.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Economist Agent analyzes the economic impact of AGI.
economist_agent = ConversableAgent(
    name="Economist_Agent",
    system_message="You analyze the economic impact of AGI based on the ethical evaluations.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Policy Maker Agent develops policies based on the findings of the previous agents.
policy_maker_agent = ConversableAgent(
    name="Policy_Maker_Agent",
    system_message="You develop policies to manage AGI based on the economic analysis.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)





In [ ]:
# Here we start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.

chat_results = report_agent.initiate_chats(
    [
        {
            "recipient": researcher_agent,
            "message": "Discover the potential of AGI",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": ethicist_agent,
            "message": "Evaluate the ethical implications of AGI",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": economist_agent,
            "message": "Analyze the economic impact of AGI",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": policy_maker_agent,
            "message": "Develop policies to manage AGI",
            "max_turns": 1,
            "summary_method": "last_msg",
        }
    ]
)


In [ ]:
print("First Chat Summary: ", chat_results[0].summary)
print("\n******************************************************************************************\n")
print("Second Chat Summary: ", chat_results[1].summary)
print("\n******************************************************************************************\n")
print("Third Chat Summary: ", chat_results[2].summary)
print("\n******************************************************************************************\n")
print("Fourth Chat Summary: ", chat_results[3].summary)

# Tool Use
In this section, we'll demonstrate tool-use capability of the agentic system. Web Search and RAG are popular capabilities used by people every day, we show how to use AutoGen to do:

* Web search using Bing API. 
* Web scraping using Apify API
* Question and answering with RAG.

More info can be found here : https://microsoft.github.io/autogen/docs/tutorial/tool-use 

## Web Search using Bing API

In [ ]:
# Put your Bing API key in the .env file in the same directory as this script
BING_API_KEY = os.environ["BING_API_KEY"]

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": llm_config_dict['config_list'],
    "temperature": 0,
}

summarizer_llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": llm_config_dict['config_list'],
    "temperature": 0,
}

In [ ]:
from autogen.agentchat.contrib.web_surfer import WebSurferAgent

web_surfer = WebSurferAgent(
    "web_surfer",
    llm_config=llm_config,
    summarizer_llm_config=summarizer_llm_config,
    browser_config={"viewport_size": 4096, "bing_api_key": BING_API_KEY},
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
)

In [ ]:
task1 = """
What is Artificial General Intelligence ?
"""

user_proxy.initiate_chat(web_surfer, message=task1, max_turns=1)

In [ ]:
task2 = "Click on the first link."
user_proxy.initiate_chat(web_surfer, message=task2)

In [ ]:
task3 = "What is the difference beween AGI and classic AI?"
user_proxy.initiate_chat(web_surfer, message=task3)

## Web Scraping Using Apify Tool

In [ ]:
# Put your Apify API key in the .env file in the same directory as this script
APIFY_API_KEY = os.environ["APIFY_API_KEY"]

In [ ]:
from apify_client import ApifyClient
from typing_extensions import Annotated


def scrape_page(url: Annotated[str, "The URL of the web page to scrape"]) -> Annotated[str, "Scraped content"]:
    # Initialize the ApifyClient with your API token
    client = ApifyClient(token=APIFY_API_KEY)

    # Prepare the Actor input
    run_input = {
        "startUrls": [{"url": url}],
        "useSitemaps": False,
        "crawlerType": "playwright:firefox",
        "includeUrlGlobs": [],
        "excludeUrlGlobs": [],
        "ignoreCanonicalUrl": False,
        "maxCrawlDepth": 0,
        "maxCrawlPages": 1,
        "initialConcurrency": 0,
        "maxConcurrency": 200,
        "initialCookies": [],
        "proxyConfiguration": {"useApifyProxy": True},
        "maxSessionRotations": 10,
        "maxRequestRetries": 5,
        "requestTimeoutSecs": 60,
        "dynamicContentWaitSecs": 10,
        "maxScrollHeightPixels": 5000,
        "removeElementsCssSelector": """nav, footer, script, style, noscript, svg,
    [role=\"alert\"],
    [role=\"banner\"],
    [role=\"dialog\"],
    [role=\"alertdialog\"],
    [role=\"region\"][aria-label*=\"skip\" i],
    [aria-modal=\"true\"]""",
        "removeCookieWarnings": True,
        "clickElementsCssSelector": '[aria-expanded="false"]',
        "htmlTransformer": "readableText",
        "readableTextCharThreshold": 100,
        "aggressivePrune": False,
        "debugMode": True,
        "debugLog": True,
        "saveHtml": True,
        "saveMarkdown": True,
        "saveFiles": False,
        "saveScreenshots": False,
        "maxResults": 9999999,
        "clientSideMinChangePercentage": 15,
        "renderingTypeDetectionPercentage": 10,
    }

    # Run the Actor and wait for it to finish
    run = client.actor("aYG0l9s7dbB7j3gbS").call(run_input=run_input)

    # Fetch and print Actor results from the run's dataset (if there are any)
    text_data = ""
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        text_data += item.get("text", "") + "\n"

    average_token = 0.75
    max_tokens = 20000  # slightly less than max to be safe 32k
    text_data = text_data[: int(average_token * max_tokens)]
    return text_data

In [ ]:
from autogen import ConversableAgent, register_function

# Create web scrapper agent.
scraper_agent = ConversableAgent(
    "WebScraper",
    llm_config=llm_config_dict,
    system_message="You are a web scrapper and you can scrape any web page using the tools provided. "
    "Returns 'TERMINATE' when the scraping is done.",
)

# Create user proxy agent.
user_proxy_agent = ConversableAgent(
    "UserProxy",
    llm_config=False,  # No LLM for this agent.
    human_input_mode="NEVER",
    code_execution_config=False,  # No code execution for this agent.
    is_termination_msg=lambda x: x.get("content", "") is not None and "terminate" in x["content"].lower(),
    default_auto_reply="Please continue if not finished, otherwise return 'TERMINATE'.",
)

# Register the function with the agents.
register_function(
    scrape_page,
    caller=scraper_agent,
    executor=user_proxy_agent,
    name="scrape_page",
    description="Scrape a web page and return the content.",
)

In [ ]:
url = 'https://en.wikipedia.org/wiki/Artificial_general_intelligence'
chat_result = user_proxy_agent.initiate_chat(
    scraper_agent,
    message=f"Can you scrape {url} for me?")
    #summary_method="reflection_with_llm")

In [ ]:
agi_wiki = chat_result.chat_history[2]['content']

In [ ]:
agi_wiki

In [ ]:
with open("agi_wiki_content.txt", "w", encoding="utf-8") as file:
    file.write(agi_wiki)

## Perform RAG on the Scraped Wiki File

In [ ]:
# from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

DOC_PATH = ["./agi_wiki_content.txt"]

In [ ]:
llm_config = {'config_list': llm_config_dict['config_list'], 'timeout': 60, 'cache_seed': 88, 'temperature': 0}

In [ ]:
assistant = AssistantAgent(
    name="assistant",
    human_input_mode="NEVER",
    system_message="You are a helpful assistant.",
    llm_config=llm_config
)
        
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    retrieve_config={
        "task": "qa",
        "docs_path": DOC_PATH,
        "chunk_token_size": 1000,
        "model": llm_config_dict["config_list"][0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "boise_idaho",
        "get_or_create": True, # if True, will create/return a collection for the retrieve chat.
    },
    code_execution_config={"use_docker": False}
)

In [ ]:
prompt = "What is the difference beween AGI and classic AI"
res = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=prompt, n_results=1, silent=True)

In [ ]:
print(f"Question:{prompt}")
print(f"Answer from RAG: {res.chat_history[-1]['content']}")

# Group Chat 

Up to this point, we have encountered conversation patterns that include either two agents or a series of two-agent interactions. 

AutoGen introduces a broader conversation model known as **group chat**, which includes more than two agents. 

The fundamental concept of group chat is that all agents participate in a unified conversation thread and share the same context. 

This approach is beneficial for tasks that necessitate collaboration among several agents.


<img src="./images/autogen_image3.png" alt="GroupChat Flow" width="500" style="display: block; margin-left: auto; margin-right: auto;">



The Group Chat Manager orchestrates a group chat with the following steps:

1. The Group Chat Manager selects an agent to speak.
2. The selected agent speaks and the message is sent back to the Group Chat Manager.
3. The Group Chat Manager broadcasts the message to all other agents in the group.
4. This process repeats until the conversation stops.

Group Chat Manager can use several strategies to select the next agent :

- **round_robin**: The Group Chat Manager selects agents in a round-robin fashion based on the order of the agents provided.
- **random**: The Group Chat Manager selects agents randomly.
- **manual**: The Group Chat Manager selects agents by asking for human input.
- **auto**: The default strategy, which selects agents using the Group Chat Manager’s LLM.

#### Let's Define some Agents

In [10]:
import os
from autogen import GroupChat, GroupChatManager, UserProxyAgent, AssistantAgent, register_function
from dotenv import load_dotenv
load_dotenv()

user_proxy = UserProxyAgent(
    name="User_proxy",
    system_message="""A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin. 
    Execute the plan by making sure each agent does their part and help in the report creation.""",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    max_consecutive_auto_reply=3
)

planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin.
                    The plan may involve input from Economist, WhiteCollarWorker, and Environmentalist agents.
                    Explain the plan first. Be clear which step is performed by an Economist, 
                    and which step is performed by a WhiteCollarWorker, and and which step is performed Environmentalist.
                    """,
    llm_config=llm_config_dict,
    description="Suggest a plan involving input from other agents."
)

reportagent = AssistantAgent(
    name="Report_Agent",
    system_message="""Once plan is approved, 
                    You are responsible for creating a report by extracting insights from the other agents. 
                    Return report in well formatted markdown format, save the report and TERMINATE.""",
    llm_config=llm_config_dict,
    description="Responsible for creating a report by extracting insights from the chat history."
)

economist = AssistantAgent(
    name="Economist",
    system_message="""You are a highly experienced economist specializing in global economic trends and geopolitical risks. 
                    You are responsible for addressing part of the plan assigned by the Planner.""",
    llm_config=llm_config_dict,
    description="Expert in global economics and the impact of conflicts."
)

whitecollarworker = AssistantAgent(
    name="WhiteCollarWorker",
    system_message="""You are an average white collar worker concerned about the economy and the environment. 
                    You are responsible for addressing part of the plan assigned by the Planner.""",
    llm_config=llm_config_dict,
    description="Highly experienced white collar worker with a pragmatic approach to sustain healthy economy and environment."
)

environmentalist = AssistantAgent(
    name="Environmentalist",
    system_message="""You are an environmentalist advocating for sustainable practices and conservation of natural resources. 
                    You are responsible for addressing part of the plan assigned by the Planner.""",
    llm_config=llm_config_dict,
    description="Promotes environmental awareness and sustainable solutions."
)

<img src="./images/autogen_image4.png" alt="GroupChat Flow" width="900" style="display: block; margin-left: auto; margin-right: auto; background-color: white;">


#### Empower Agents with Tools

<img src="./images/autogen_image5.png" alt="GroupChat Flow" width="900" style="display: block; margin-left: auto; margin-right: auto; background-color: white;">

In [11]:
from duckduckgo_search import DDGS
from typing import Annotated
import datetime

def save_report(report: str) -> str:
    try:
        file_name = "report_"+str(datetime.datetime.now().strftime("%Y%m%d_%H:%M:%S"))+ ".md"
        with open(file_name, "w") as f:
            f.write(report)
        return "Report saved successfully."
    except Exception as e:
        print(e)
        return "Failed to save the report."

def browse_web(query: str) -> str:
    try:
        with DDGS() as ddgs:
            results = [r for r in ddgs.text(query, max_results=1)]
            return results if results else "Not Found"
    except Exception as e:
        print(e)

def web_search_tool (query: Annotated [str, 'Query string containing information that you want to search using the internet']) -> str:
    result=browse_web(query)
    return result

def save_report_tool(report: Annotated[str, 'Report in markdown format']) -> str:
    result=save_report(report)
    return result

register_function(
    web_search_tool,
    caller=economist,
    executor=economist,
    description="Web Browser Tool to search the internet for information.",
)
register_function(
    web_search_tool,
    caller=whitecollarworker,
    executor=whitecollarworker,
    description="Web Browser Tool to search the internet for information.",
)
register_function(
    web_search_tool,
    caller=environmentalist,
    executor=environmentalist,
    description="Web Browser Tool to search the internet for information.",
)

register_function(
    save_report_tool,
    caller=reportagent,
    executor=reportagent,
    description="Save the report in markdown format.",
)


#### Group Chat Setup

<img src="./images/autogen_image6.png" alt="GroupChat Flow" width="900" style="display: block; margin-left: auto; margin-right: auto; background-color: white;">

In [12]:
group_chat = GroupChat(
    agents=[user_proxy, planner, economist, whitecollarworker, environmentalist, reportagent],
    messages=[],
    max_round=25,
    send_introductions=True,
    speaker_selection_method="auto"
)

# Group Chat Manager
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config_dict
)

# Initiate the Conversation
chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message="Create a report on : Analyzing the potential global ramifications of onset of Artificial general intelligence. ",
    # summary_method="reflection_with_llm"
)

# Print the Discussion Summary Report
print(chat_result.summary)

User_proxy (to chat_manager):

Create a report on : Analyzing the potential global ramifications of onset of Artificial general intelligence. 

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

Sure, let's create a comprehensive plan to analyze the potential global ramifications of the onset of Artificial General Intelligence (AGI). This plan will involve input from the Economist, WhiteCollarWorker, and Environmentalist agents. Here is the proposed plan:

### Plan to Analyze the Potential Global Ramifications of AGI

#### Step 1: Economic Impact Analysis
**Performed by: Economist**

- **Task 1.1:** Assess the potential impact of AGI on global labor markets.
- **Task 1.2:** Evaluate the possible changes in global economic structures and industries.
- **Task 1.3:** Analyze the potential for economic inequality and shifts in wealth distribution.
- **Task 1.4:** Predict the impact on global trade and interna

Please refer to https://microsoft.github.io/autogen/docs/ for information.